In [1]:
import asyncio, re, pathlib
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

APP_URL = "https://indopremier.com/#ipot/app"
OUTDIR = pathlib.Path("artifacts"); OUTDIR.mkdir(exist_ok=True)

# ---------- helpers aman ----------
def alive(page):
    try:
        return (page is not None) and (not page.is_closed())
    except:
        return False

async def click_if_exists(loc):
    try:
        if await loc.count():
            await loc.first.click(timeout=1200)
            return True
    except Exception:
        pass
    return False

async def close_cookie_banner(page):
    """OneTrust / banner umum, klik tombol di dalam bannernya saja."""
    if not alive(page): return
    try:
        # OneTrust
        banner = page.locator("#onetrust-banner-sdk, [id*='onetrust']")
        if await banner.is_visible(timeout=800):
            for sel in [
                "#onetrust-accept-btn-handler",
                "button:has-text('Accept')",
                "button:has-text('Setuju')",
                "button:has-text('OK')",
            ]:
                if await click_if_exists(banner.locator(sel)):
                    return
    except: pass

async def close_modals(page, rounds=5):
    """Tutup modal/tooltip dengan hati2: hanya klik BUTTON di dalam container modal."""
    if not alive(page): return
    BTN_TXT = re.compile(r"(Got ?It|OK|Oke|Okay|Setuju|Close|Tutup|Dismiss|Skip|Lewati|Saya Mengerti|Mengerti|Done|Finish|Lanjut|Continue|Next)", re.I)
    CONTAINERS = [
        "[role='dialog']",
        ".ant-modal-root .ant-modal", ".ant-drawer", ".ant-message", ".ant-notification",
        ".MuiDialog-container .MuiDialog-paper", ".MuiDrawer-root", ".MuiSnackbar-root",
        ".modal:visible", "[data-modal]", "[class*='modal']",
        ".introjs-overlay", "[data-shepherd-step]", "[class*='tour']",
        "[class*='tooltip']"
    ]
    for _ in range(rounds):
        touched = False
        if not alive(page): break

        # ESC sering menutup modal
        try:
            await page.keyboard.press("Escape")
            touched = True
        except: pass

        # Tutup banner cookie dulu (kalau ada)
        await close_cookie_banner(page)

        # Tutup tiap container modal yang terlihat
        for cont_sel in CONTAINERS:
            if not alive(page): break
            cont = page.locator(cont_sel)
            try:
                if await cont.count():
                    vis = await cont.first.is_visible()
                else:
                    vis = False
            except:
                vis = False
            if not vis: continue

            # tombol close khusus
            for sel in [
                "button[aria-label*='close' i]",
                "button[aria-label*='tutup' i]",
                ".ant-modal-close button",
                ".MuiDialog-root [aria-label*='close' i]",
                "button:has-text('×')",
            ]:
                if await click_if_exists(cont.locator(sel)): touched = True

            # tombol aksi teks (OK/Tutup/Skip/dll) — hanya BUTTON
            try:
                btns = cont.get_by_role("button", name=BTN_TXT)
                while alive(page) and await btns.count():
                    if await click_if_exists(btns):
                        touched = True
                    else:
                        break
            except: pass

            # backdrop (klik di area modal/backdrop saja)
            for sel in [".MuiBackdrop-root", ".ant-modal-wrap", ".ant-drawer-mask", "[class*='backdrop']"]:
                try:
                    bd = page.locator(sel)
                    if await bd.is_visible():
                        await bd.click(timeout=500)
                        touched = True
                except: pass

        if not touched:
            break
        await asyncio.sleep(0.2)

# ---------- main ----------
async def main():
    async with async_playwright() as p:
        ctx = await p.chromium.launch_persistent_context(
            user_data_dir=str(OUTDIR / "ipot-desktop-profile"),
            headless=False,
            viewport={"width":1600,"height":900},
            args=["--disable-blink-features=AutomationControlled"],
            locale="id-ID",
            timezone_id="Asia/Jakarta",
        )
        page = ctx.pages[0] if ctx.pages else await ctx.new_page()

        print("→ Opening:", APP_URL)
        try:
            await page.goto(APP_URL, wait_until="domcontentloaded", timeout=60000)
        except PWTimeout:
            print("! goto timeout — lanjut.")

        # beri waktu komponen SPA render + popup nongol
        await asyncio.sleep(2.0)

        # bersihkan popup dengan selector ketat (hanya di container modal)
        await close_modals(page)

        if not alive(page):
            print("! Page closed by site/user. Stop.")
            return

        # klik tombol panel member area (persis sesuai spek)
        try:
            panel_btn = page.locator('a.panel-open[data-panel=".panel-memberarea"]')
            await panel_btn.wait_for(state="visible", timeout=10000)
            await panel_btn.click()
            print("✓ Panel member area dibuka.")
        except Exception as e:
            print("! Gagal klik panel member area:", e)
            # coba ulang setelah sapu modal sekali lagi
            await close_modals(page)
            await panel_btn.click(timeout=8000)

        # klik link Login di panel (persis selector yang kamu kasih)
        try:
            login_link = page.locator('a.login-href.panel-close[href="https://indopremier.com/login/"]')
            await login_link.wait_for(state="visible", timeout=10000)
            await login_link.click()
            print("✓ Klik link Login.")
        except Exception as e:
            print("! Gagal klik Login:", e)
            # fallback minimal berdasarkan text
            try:
                await page.get_by_role("link", name=re.compile(r"Login", re.I)).first.click(timeout=6000)
                print("✓ Klik Login (fallback).")
            except Exception as e2:
                print("! Fallback Login gagal:", e2)

        # tunggu pindah ke halaman login (kalau berhasil)
        try:
            await page.wait_for_url("**/login/**", timeout=15000)
        except PWTimeout:
            pass

        # bukti
        try:
            snap = OUTDIR / "after_click_login.png"
            await page.screenshot(path=str(snap), full_page=True)
            print("✓ Screenshot:", snap)
        except Exception as e:
            print("! screenshot error:", e)

        print("⏸ Standby; biarkan cell berjalan agar window tetap terbuka.")
        while True:
            await asyncio.sleep(3600)

await main()


→ Opening: https://indopremier.com/#ipot/app
✓ Panel member area dibuka.
! Gagal klik Login: Locator.wait_for: Timeout 10000ms exceeded.
Call log:
  - waiting for locator("a.login-href.panel-close[href=\"https://indopremier.com/login/\"]") to be visible

✓ Klik Login (fallback).
✓ Screenshot: artifacts/after_click_login.png
⏸ Standby; biarkan cell berjalan agar window tetap terbuka.


CancelledError: 